<a href="https://colab.research.google.com/github/Omkar2499/Dataset/blob/main/Segmentation_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip -q Processed_18_classes.zip

In [2]:
import os
import glob

# Specify the path to the main directory
main_dir = '/content/Output_Images'

# Get a list of all subdirectories
subdirs = [os.path.basename(x) for x in glob.glob(os.path.join(main_dir, '*')) if os.path.isdir(x)]

# Join the subdirectory names into a single string with commas and double quotes
result = '", "'.join(subdirs)

# Add double quotes at the beginning and end of the string
result = '"' + result + '"'

# Print the result
print(result)


"Slush", "asphalt_dry", "Asphalt_wet", "Black_Ice", "Heavy_Snow", "Light Snow", "Cobblestone_wet", "Muddy_wet", "Dry Mud", "Refrozen Ice", "gravel_dry", "Sleet", "Gravel_wet", "Concrete_wet", "concrete_dry", "Heavy_wet", "Packed Snow", "cobblestone_dry"


In [3]:
import tensorflow as tf

import os

import shutil

import numpy as np

from sklearn.model_selection import train_test_split

classes = ["Slush", "asphalt_dry", "Asphalt_wet", "Black_Ice", "Heavy_Snow", "Light Snow", "Cobblestone_wet", "Muddy_wet", "Dry Mud", "Refrozen Ice", "gravel_dry", "Sleet", "Gravel_wet", "Concrete_wet", "concrete_dry", "Heavy_wet", "Packed Snow", "cobblestone_dry"

]



data_dir = '/content/Output_Images'

train_dir = '/content/train'

test_dir = '/content/test'

# Create train and test directories

os.makedirs(train_dir, exist_ok=True)

os.makedirs(test_dir, exist_ok=True)

for cls in classes:

    # Create class-specific train and test directories

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)

    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

    # Get a list of all image files for this class

    image_files = os.listdir(os.path.join(data_dir, cls))

    # Split the files into train and test sets

    train_files, test_files = train_test_split(image_files, test_size=0.3, random_state=42)  # 80% train, 20% test

    # Copy the train files into the train directory

    for file in train_files:

        shutil.copy(os.path.join(data_dir, cls, file), os.path.join(train_dir, cls, file))

    # Copy the test files into the test directory

    for file in test_files:

        shutil.copy(os.path.join(data_dir, cls, file), os.path.join(test_dir, cls, file))

In [4]:
import tensorflow as tf

from tensorflow.keras.applications import VGG16

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.optimizers import Adam

import scipy

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# Define the data generators

train_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(

    '/content/train',

    target_size=(224, 224),

    batch_size=32,

    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(

    '/content/test',

    target_size=(224, 224),

    batch_size=32,

    class_mode='categorical')

Found 1258 images belonging to 18 classes.
Found 540 images belonging to 18 classes.


In [5]:

baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=tf.keras.Input(shape=(224, 224, 3)))

# Construct the head of the model that will be placed on top of the base model

headModel = baseModel.output

headModel = Flatten(name="flatten")(headModel)

headModel = Dense(512, activation="relu")(headModel)

headModel = Dense(18, activation="softmax")(headModel)  # Change the number here according to your number of classes

# Place the head FC model on top of the base model (this will become the actual model we will train)

model = tf.keras.Model(inputs=baseModel.input, outputs=headModel)

# Loop over all layers in the base model and freeze them so they will not be updated during the training process

for layer in baseModel.layers:

    layer.trainable = False

# Compile the model

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

model_checkpoint = ModelCheckpoint('processed_images_model.keras', save_best_only=True, monitor='val_loss')

model.fit(train_generator, validation_data=test_generator, epochs=10, callbacks=[early_stopping, model_checkpoint])

Epoch 1/10


In [ ]:
import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns



# Load the test data

test_generator = test_datagen.flow_from_directory(

    '/content/test',

    target_size=(224, 224),

    batch_size=32,

    class_mode='categorical',

    shuffle=False)

# Make predictions

predictions = model.predict(test_generator)

predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

print(predicted_classes)
# Generate the confusion matrix

conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix

plt.figure(figsize=(10, 8))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)

plt.xlabel('Predicted')

plt.ylabel('True')

plt.title('Confusion Matrix')

plt.show()

# Print classification report

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

In [11]:
import cv2
image = cv2.imread('/content/test/gravel_dry/gravel_dry_1.png')
image = cv2.resize(image,(224,224))
normalized = image / 255.0


frame_array = np.array([normalized])
    # predictions = model1.predict(frame_array)
    # class_id = np.argmax(predictions)
    # predicted_class = class_n[class_id]

predictions1 = model.predict(frame_array)
class_id = np.argmax(predictions1)
predicted_class1 = classes[class_id]

predicted_class_indices = np.argsort(predictions1[0])[::-1][:3]  # Indices of top 3 probabilities
predicted_class_names = [classes[index] for index in predicted_class_indices]
confidences = [predictions1[0, index] for index in predicted_class_indices]

predicted_class_names

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


['gravel_dry', 'Gravel_wet', 'Black_Ice']

In [17]:
input_video_path = '/content/updated.mp4'
output_video_path = 'updated_output.mp4'
dashboard_image_path = '/content/dash.png'  # Dashboard path



classes = ["Cobblestone_wet", "asphalt_dry", "Black_Ice", "Refrozen Ice", "concrete_dry", "Dry Mud",
           "Asphalt_wet", "Concrete_wet", "Heavy_wet", "cobblestone_dry", "Muddy_wet", "Packed Snow",
           "Slush", "Sleet", "Heavy_Snow", "Gravel_wet", "Light Snow", "gravel_dry"
]

friction_coefficients = {
    "Asphalt_wet" :"0.40-0.50",
    "Black_Ice": "0.005-0.02",
    "Cobblestone_wet" : "0.30-0.40",
    "Concrete_wet" : "0.40-0.50",
    "Gravel_wet" : "0.30-0.40",
    "Heavy_Snow": "0.15-0.30",
    "Heavy_wet" :  "0.30-0.50",
    "Light Snow": "0.05-0.20",
    "Muddy_wet" :   "0.20-0.30",
    "Packed Snow": "0.05-0.15",
    "Refrozen Ice": "0.30-0.40",
    "Sleet": "0.50",
    "Slush": "0.35-0.40",
    "asphalt_dry" : "0.70-0.80",
    "cobblestone_dry" : "0.60-0.70",
    "concrete_dry" :  "0.70-0.80",
    "Dry Mud" :  "0.50-0.60",
    "gravel_dry" : "0.60-0.70"
}




# Load the dashboard image
dashboard = cv2.imread(dashboard_image_path, cv2.IMREAD_UNCHANGED)

# Check if the image has 4 channels (RGBA)
if dashboard.shape[-1] == 4:
    # Convert the image from RGBA to RGB
    dashboard = cv2.cvtColor(dashboard, cv2.COLOR_BGRA2BGR)

# Resize the dashboard to fit the video frame (adjust dimensions as needed)
dashboard_height, dashboard_width = 150, 300  # Adjust the size if needed
dashboard = cv2.resize(dashboard, (dashboard_width, dashboard_height))





cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


x_offset = (frame_width - dashboard_width) // 2
y_offset = 0  # or adjust this to your desired top margin

# Font settings for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.27
font_color = (255, 255, 255)  # White text
thickness = 1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Overlay the dashboard at the top-left corner of the frame
    frame[0:dashboard_height, 0:dashboard_width] = dashboard

    # Overlay the dashboard at the top-center of the frame
    #x_offset = (frame_width - dashboard_width) // 2
    #frame[0:dashboard_height, x_offset:x_offset + dashboard_width] = dashboard

    # Prediction road surface type for a frame
    resized_frame = cv2.resize(frame, (224,224))
    normalized_frame = resized_frame / 255.0
    frame_array = np.array([normalized_frame])
    predictions = model.predict(frame_array)
    predicted_class_indices = np.argsort(predictions[0])[::-1][:3]
    predicted_class_names = [classes[index] for index in predicted_class_indices]
    predicted_class = predicted_class_names[0]
    confidences = [predictions[0, index] for index in predicted_class_indices]



        # Calculate w values
    mu_values = []
    for class_name in predicted_class_names:
        mu_range = friction_coefficients[class_name].split('-')
        mu = (float(mu_range[0]) + float(mu_range[1])) / 2 if len(mu_range) == 2 else float(mu_range[0])
        mu_values.append(mu)

    sum_confidences = sum(confidences)
    friction = sum((conf / sum_confidences) * mu for conf, mu in zip(confidences, mu_values))
    # friction_EKF = [kalman_filter(mu) for mu in mu_values]
    # surface_friction_EKF = f'{friction_EKF[0]:.2f}'
    surface_friction = f'{friction:.2f}'

    # Assign Slip Mu values to each wheel
    slip_mu_values = [surface_friction, surface_friction, surface_friction, surface_friction]

    # Add predictions and Slip Mu values to the dashboard
    #WFL
    cv2.putText(
        frame,
        f"WFL: {predicted_class}",
        (10, 20),  # Top-left corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {slip_mu_values[0]}",
        (33, 23 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    # WFR
    cv2.putText(
        frame,
        f"WFR: {predicted_class}",
        (dashboard_width - 150, 20),  # Top-right corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {slip_mu_values[1]}",
        (dashboard_width - 107, 23 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    # WRL
    cv2.putText(
        frame,
        f"WRL: {predicted_class}",
        (12, dashboard_height - 20),  # Bottom-left corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {slip_mu_values[2]}",
        (35, dashboard_height - 17 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    # WRR
    cv2.putText(
        frame,
        f"WRR: {predicted_class}",
        (dashboard_width - 150, dashboard_height - 20),  # Bottom-right corner of the dashboard
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )
    cv2.putText(
        frame,
        f"Mu: {slip_mu_values[3]}",
        (dashboard_width - 107, dashboard_height - 17 + int(font_scale * 25)),  # Adjusted y-coordinate for the second line
        font, font_scale, font_color, thickness, cv2.LINE_AA
    )











    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()

print(f'Output video saved to: {output_video_path}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━